In [31]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [33]:
df=pd.read_csv("IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [35]:
print(df.columns)

Index(['review', 'sentiment'], dtype='object')


In [37]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [39]:
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)


In [41]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


In [49]:
max_len = 100  
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=max_len),
    GRU(64),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

C:\Users\lenovo\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [51]:
history = model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_split=0.2)

y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.7108 - loss: 0.5337 - val_accuracy: 0.8635 - val_loss: 0.3142
Epoch 2/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.9037 - loss: 0.2422 - val_accuracy: 0.8685 - val_loss: 0.3081
Epoch 3/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.9445 - loss: 0.1545 - val_accuracy: 0.8608 - val_loss: 0.3311
Epoch 4/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.9677 - loss: 0.0979 - val_accuracy: 0.8543 - val_loss: 0.4126
Epoch 5/5
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.9794 - loss: 0.0640 - val_accuracy: 0.8453 - val_loss: 0.5623
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Test Accuracy: 0.8442


In [53]:
print("\n Sample Predictions:")
for i in range(5):
    print(f"Review: {X_test.iloc[i][:100]}...")
    print(f"Actual Sentiment: {y_test.iloc[i]} - Predicted Sentiment: {y_pred[i]}")
    print("-" * 50)


 Sample Predictions:
Review: I really liked this Summerslam due to the look of the arena, the curtains and just the look overall ...
Actual Sentiment: 1 - Predicted Sentiment: 0
--------------------------------------------------
Review: Not many television shows appeal to quite as many different kinds of fans like Farscape does...I kno...
Actual Sentiment: 1 - Predicted Sentiment: 1
--------------------------------------------------
Review: The film quickly gets to a major chase scene with ever increasing destruction. The first really bad ...
Actual Sentiment: 0 - Predicted Sentiment: 0
--------------------------------------------------
Review: Jane Austen would definitely approve of this one!<br /><br />Gwyneth Paltrow does an awesome job cap...
Actual Sentiment: 1 - Predicted Sentiment: 1
--------------------------------------------------
Review: Expectations were somewhat high for me when I went to see this movie, after all I thought Steve Care...
Actual Sentiment: 0 - Predicted Se